In [ ]:
import sys
sys.path.append("../..")
from argparse import ArgumentParser
from tqdm import tqdm
import numpy as np
from PIL import Image
from io import BytesIO
from os import makedirs
from os.path import join
import cv2 as cv
import numpy as np
from os.path import basename, splitext
from girder_client import GirderClient


def login(
    apiurl: str, username: str = None, password: str = None
) -> GirderClient:
    """Authenticate girder client instance.
    
    Args:
        apiurl: API URL.
        username: Username.
        password: Password.
    
    Returns:
        gc: Girder client.
        
    """
    gc = GirderClient(apiUrl=apiurl)

    if username is None or password is None:
        interactive = True
    else:
        interactive = False

    gc.authenticate(username=username, password=password, 
                    interactive=interactive)

    return gc


def imwrite(fp: str, img: np.ndarray, grayscale: bool = False):
    """Write image to file.
    
    Args:
        fp: Filepath to save image.
        img: Image to save.
        grayscale: True to save image as a grayscale image, otherwise it is
            saved as an RGB image.
    
    """
    if grayscale:
        cv.imwrite(fp, img)
    else:
        cv.imwrite(fp, cv.cvtColor(img, cv.COLOR_RGB2BGR))

def get_filename(fp: str, prune_ext: bool = True) -> str:
    """Get the filename of a filepath.

    Args:
        fp: Filepath.
        prune_ext: Remove extension.
    
    Returns:
        Filename.

    """
    fn = basename(fp)

    if prune_ext:
        fn = splitext(fn)[0]

    return fn

def get_tile_metadata(gc: GirderClient, item_id: str) -> dict:
    """Get the tile source metadata for an item with a large image
    associated with it.

    Args:
        gc: Girder client.
        item_id: DSA WSI id.

    Returns:
        Metadata for large image associated.

    """
    return gc.get(f"item/{item_id}/tiles")


In [ ]:
"""This function will work for jupyter notebook but needs to be modified if running a python
def parse_args():
    """CLI arguments."""
    parser = ArgumentParser()

    parser.add_argument("--user", type=str, default=None, help="DSA username.")
    parser.add_argument("--password", type=str, default=None, help="DSA password.")
    parser.add_argument(
        "--fld-id",
        type=str,
        default="650887979a8ab9ec771ba678",
        help="DSA folder ID with images of interest.",
    )
    parser.add_argument(
        "--save-dir", type=str, default=".", help="Location to save images."
    )
    parser.add_argument(
        "--api-url",
        type=str,
        help="DSA API URL.",
        default="http://glasslab.neurology.emory.edu:8080/api/v1",
    )

    args, unknown = parser.parse_known_args()
    return args



In [2]:
def main():
    """Main function."""
    args = parse_args()

    # Authenticate client.
    gc = login(args.api_url, username=args.user, password=args.password)

    # Create location to save images.
    makedirs(args.save_dir, exist_ok=True)

    # Loop through all the images.
    for item in tqdm(list(gc.listItem(args.fld_id))):
        # Read the metadata to identify the nuclei/DAPI channel.
        channels = item.get("meta", {}).get("Channels", {})

        # Look for nuclei channel.
        channel = None

        for k, v in channels.items():
            if v == "Nuclei":
                channel = k
                break

        # Skip image if no nuclei channel.
        if channel is None:
            continue

        if channel.startswith("Channel"):
            # Special case where channels were not named.
            frame = int(channel[-1]) - 1  # get the frame
        else:
            # Identify the frame that contains nuclei image.
            channel_map = get_tile_metadata(gc, item["_id"]).get("channelmap", {})

            frame = channel_map[channel]

        # Get the image by frame.
        response = gc.get(
            f"item/{item['_id']}/tiles/region?units=base_pixels&exact="
            + f"false&frame={frame}&encoding=PNG",
            jsonResp=False,
        )

        # Save images.
        img = np.array(Image.open(BytesIO(response.content)))

        imwrite(join(args.save_dir, f"{get_filename(item['name'])}.png"), img)


In [7]:
if __name__ == "__main__":
    main()